In [1]:
import torch
torch.cuda.is_available(), torch.cuda.is_bf16_supported()

(True, True)

In [2]:
DEVICE = "cuda" if torch.cuda.is_available else "cpu"

In [3]:
seq_len = 512
head_size = 32

In [ ]:
q = torch.randn(seq_len, head_size)
k = torch.randn(seq_len, head_size)
v = torch.randn(seq_len, head_size)

In [24]:
torch.arange(B_r * 63, B_r * 64)

tensor([504, 505, 506, 507, 508, 509, 510, 511])

In [ ]:
from math import sqrt
M = 1024
# Block col size
B_c = M//(4 * head_size)
B_r = min(M//(4 * head_size), head_size)

O = torch.zeros(seq_len, head_size)
l = torch.zeros(seq_len)
m = torch.fill(torch.zeros(seq_len), float('-inf'))
# mask = torch.tril(torch.ones(B_r, B_c)).to(DEVICE)

# iterate over seqlen/B_c blocks of K and V
T_c = seq_len // B_c
T_r = seq_len // B_r
for j in range(T_c):

    # (B_c x head_size)
    Kj = k[B_c * j:B_c * (j+1), :].to(DEVICE)
    Vj = v[B_c * j:B_c * (j+1), :].to(DEVICE)

    k_idx = torch.arange(B_c * j, B_c * (j + 1)).unsqueeze(0)

    for i in range(T_r):
        if i < j:
            continue

        # (B_r x head_size)
        Qi = q[B_r * i:B_r * (i+1), :].to(DEVICE)
        scale = 1.0 / sqrt(head_size)

        # (B_r x head_size) @ (head_size x B_c)
        # (B_r x B_c)
        Sij = (Qi @ Kj.T) * scale

        q_idx = torch.arange(B_r * i, B_r * (i + 1)).unsqueeze(1)
        # B_r x B_c
        mask = (q_idx >= k_idx).to(DEVICE)

        # THE FUCKING BUG WAS HERE BUG BUG BUG FUCKKKKK
        # here's a tip, heer. masked_fill is not in-place. masked_fill_ is
        
        Sij.masked_fill_(~mask, float('-inf'))
        # (B_r)
        mi_old = m[B_r * i : B_r * (i + 1)].to(DEVICE)

        # (B_r)
        li_old = l[B_r * i : B_r * (i + 1)].to(DEVICE)

        # (B_r x head_size)
        oi_old = O[B_r * i : B_r * (i + 1), :].to(DEVICE)

        # torch max returns max of whole matrix if dim not specified.
        # dim=1 searches for max across rows and along columns 
        # 0th index has the max values and 1st index has the indices of max values
        # (B_r)
        mij = torch.max(Sij,dim=1)[0]
                
        # (B_r x B_c) - (B_r) would be broadcasted to (B_r x B_r) and applied row wise
        # unsqueeze(1) will make it (B_r x 1) and then it will be broadcasted column wise.
        Pij = torch.exp(Sij - mij.unsqueeze(1))
        lij = torch.sum(Pij, dim=1)

        # stacked shape (2, B_r)
        # max needs to be (B_r)
        # reduce axis 0?
        mi_new = torch.maximum(mi_old, mij).to(DEVICE)
        
        li_new = (torch.exp(mi_old - mi_new) * li_old + torch.exp(mij - mi_new) * lij).to(DEVICE)
        # print(Sij.shape, Vj.shape)
        # print(mi_old.shape)
        # print(oi_old.shape)
        # print(li_old.shape)
        # print(torch.diag(li_old).shape)
        # print(torch.exp(mi_old - mi_new).shape)
        old_part = (oi_old * li_old.unsqueeze(1)) * torch.exp(mi_old - mi_new).unsqueeze(1)
        # print(old_part.shape)
        new_part = Pij @ Vj * (torch.exp(mij - mi_new).unsqueeze(1))
        # print(new_part.shape)
        # print(torch.diag(li_new).inverse())
        Oi_new = ((old_part + new_part) / li_new.unsqueeze(1)).to(DEVICE)
        # print(Oi_new)
        O[B_r * i : B_r * (i + 1), :] = Oi_new 
        m[B_r * i : B_r * (i + 1)] = mi_new
        l[B_r * i : B_r * (i + 1)] = li_new
        Qi.cpu()
        del mi_old
        del oi_old
        del li_old
        del Oi_new
        del mi_new
        del li_new
    Kj.cpu()
    Vj.cpu()


In [50]:
O

tensor([[ 0.3456, -0.1381,  0.8336,  ...,  0.7394,  0.9866,  1.4422],
        [-0.1640,  0.3329,  0.5074,  ...,  1.1770,  0.4180,  0.8048],
        [-0.3569,  0.6638,  0.4302,  ...,  0.7971,  0.4022,  0.3649],
        ...,
        [-0.0224, -0.0142,  0.0565,  ..., -0.0179, -0.0305, -0.0751],
        [-0.0249, -0.0521,  0.0745,  ..., -0.0172,  0.0174, -0.0564],
        [ 0.0767,  0.0471, -0.0025,  ..., -0.0499,  0.0153,  0.0019]])

In [52]:
attn = torch.nn.functional.scaled_dot_product_attention(q, k, v, is_causal=True,scale=1)
attn

tensor([[ 0.3456, -0.1381,  0.8336,  ...,  0.7394,  0.9866,  1.4422],
        [ 0.1685,  0.0256,  0.7203,  ...,  0.8915,  0.7890,  1.2207],
        [-0.1261,  0.3014,  0.5328,  ...,  1.1316,  0.4651,  0.8476],
        ...,
        [-0.4891,  0.2060, -0.0738,  ..., -1.4355, -1.0461,  0.0379],
        [ 0.6193, -0.5578,  0.1044,  ...,  0.2945,  0.2020,  0.2111],
        [ 1.2384, -0.2218, -0.0946,  ...,  0.1298,  0.2802,  0.6814]])